### Notebook to create the octree of a pointcloud to be used in the potree viewer.
### It uses version 2.0 of the potree converter

### We create one massive octree, not one per gene (which was the previous workplan)

In [1]:
import pandas as pd
import numpy as np
import laspy
import os
import subprocess

In [2]:
df = pd.read_csv(r"E:\OMNI-SCI-FV\MsBrain_Eg1_VS6_JH_V6_05-02-2021\region_0\detected_transcripts.csv")

In [3]:
glyphs = sorted(['star6', 'star5', 'diamond', 'square', 'triangleUp', 'triangleDown', 'triangleRight', 'triangleLeft', 'cross', 'plus', 'asterisk', 'circle', 'point'])
glyphs

['asterisk',
 'circle',
 'cross',
 'diamond',
 'plus',
 'point',
 'square',
 'star5',
 'star6',
 'triangleDown',
 'triangleLeft',
 'triangleRight',
 'triangleUp']

In [4]:
# Colors must be between 0 and 255
df_colors = pd.read_csv('merfish_colour_scheme.csv')
df = df.merge(df_colors, on='gene')

In [5]:
genes, gene_id = np.unique(df.gene, return_inverse=True)
df['gene_id'] = gene_id
df['classification'] = gene_id
df['pointSourceID'] = df.gene_id % len(glyphs) #source id is the gly

In [6]:
## remove the mean from xyz coords. Actually I should be removing the width/2, height/2 and depth/2
## but this is a good enough approximation
df.global_x = df.global_x - df.global_x.mean()
df.global_y = df.global_y - df.global_y.mean()
df.global_z = df.global_z - df.global_z.mean()

In [7]:
def make_las(df, las_path):
    raw = df.values
    xyz = np.ascontiguousarray(raw[:, 3:6], dtype='float32')
    rgb = np.ascontiguousarray(raw[:, 10:13], dtype='float32')
    classification = np.ascontiguousarray(raw[:, -2], dtype='float32')
    pointSourceID = np.ascontiguousarray(raw[:, -1], dtype='float32')
    
    hdr = laspy.LasHeader(version="1.4", point_format=7)
    mins = np.floor(np.min(xyz, axis=0))
    # mins = [352, 6126, 0]
    hdr.offset = mins
    hdr.scales = np.array([0.001, 0.001, 0.001])

    # 2. Create a Las
    las = laspy.LasData(hdr)

    las.x = xyz[:,0]
    las.y = xyz[:,1]
    las.z = xyz[:,2]
    las.red = rgb[:,0] 
    las.green = rgb[:,1]
    las.blue = rgb[:,2]
    las.classification = classification
    las.pt_src_id = pointSourceID
    # las.intensity = i
    
    # out_path = r"gene_pointclouds_z_spacing_1.5micron/las/%s.las" % gene
    out_filename = os.path.join(las_path, 'merfish.las')
    if not os.path.exists(os.path.dirname(out_filename)):
       os.makedirs(os.path.dirname(out_filename))
    las.write(out_filename)
    print('las file saved at: %s ' % out_filename) 

In [8]:
def make_octree(las_path):
    exe = r"F:\potree\PotreeConverter_windows_x64\PotreeConverter.exe"
    output_dir = os.path.join(os.path.dirname(las_path), 'octree', 'merfish')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    result = subprocess.run([exe, las_path, "-o", output_dir, "-m", "poisson"], capture_output=True, shell=True)
    print('octree saved at: %s ' % output_dir) 


In [9]:
# Set here where the las files will be saved. The octrees will be under the same parent dir
las_dir = os.path.join('F:\\' 'potree', 'merfish_pointcloud', 'las')

# make now the octree 
# mask = df.classification < 10
make_las(df, las_dir)
make_octree(las_dir)

las file saved at: F:\potree\merfish_pointcloud\las\merfish.las 
octree saved at: F:\potree\merfish_pointcloud\octree\merfish 


In [10]:
df.pointSourceID.max()

12